Received message:     qwered 
Received message:     qwered 
Received message:     qwered 


In [ ]:

## Читаем с автоматическим офсетом
from confluent_kafka import Consumer, KafkaException
# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group11wfewf12',
    'auto.offset.reset': 'earliest',  # Чтение с самого начала, если офсеты не найдены
    'enable.auto.commit': True        # Включение автоматического коммита офсетов
}
# Создание консюмера
consumer = Consumer(conf)
# Подписка на топик
consumer.subscribe(['my-avro-topic'])
try:
    for _ in range(5):
        msg = consumer.poll(timeout=1.0)  # Ожидание новых сообщений  
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        print(f'Received message: {msg.value().decode("utf-8")}')
        
finally:
    # Закрытие консюмера
    consumer.close()


## Читаем с РУЧНЫМ офсетом

In [76]:

from confluent_kafka import Consumer, KafkaException
# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group-222222222', ## другая группа
    'auto.offset.reset': 'earliest',  # Чтение с самого начала, если офсеты не найдены
    'enable.auto.commit': False        # 
}
# Создание консюмера
consumer = Consumer(conf)
# Подписка на топик
consumer.subscribe(['my-avro-topic'])
try:
    for _ in range(15):
        msg = consumer.poll(timeout=1.0)  # Ожидание новых сообщений  
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        print(f'Received message: {msg.value().decode("utf-8")}')
        
finally:
    # Закрытие консюмера
    consumer.close()

Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 


Чтение офсетов для конкретной группы консумеров
Для чтения текущих офсетов для группы консумеров можно использовать метод committed(partitions). Этот метод возвращает список офсетов для указанных разделов (partitions).

In [33]:
from confluent_kafka import Consumer, TopicPartition

# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group11wfewf',
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': False
}

consumer = Consumer(conf)
consumer.subscribe(['my-topic'])

# Получение текущих офсетов для группы консумеров по разделам топика
partitions = [TopicPartition('my-topic', partition=0)]
offsets = consumer.committed(partitions)

for tp in offsets:
    print(f'Partition: {tp.partition}, Offset: {tp.offset}')

consumer.close()


Partition: 0, Offset: 9


### Producer


In [34]:
from confluent_kafka import Producer

# Функция для обработки успешной доставки сообщений
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

# Конфигурация продюсера
conf = {
    'bootstrap.servers': 'localhost:9092',  # Адрес Kafka брокера
    'client.id': 'my-producer'                # Идентификатор клиента
}

# Создание экземпляра продюсера
producer = Producer(conf)

# Отправка сообщений
try:
    for i in range(4):
        key = f'key-{i}'                # Ключ сообщения (опционально)
        value = f'Hello Kafka {i}'      # Значение сообщения
        producer.produce('my-topic', key=key, value=value, callback=delivery_report)

        # Обработка асинхронных операций
        producer.poll(0)  # Проверка на наличие ошибок и вызов обратных функций

    # Ожидание отправки всех сообщений
    producer.flush()

except Exception as e:
    print(f'Error while producing: {e}')

finally:
    # Закрытие продюсера
    print('finally')


Message delivered to my-topic [0] at offset 9
Message delivered to my-topic [0] at offset 10
Message delivered to my-topic [0] at offset 11
Message delivered to my-topic [0] at offset 12
finally


### exactly-once semantics, EOS

 idempotent producer и настроек для транзакций
 Конфигурация продюсера:

enable.idempotence: True: Включает идемпотентный режим для продюсера.
transactional.id: Уникальный идентификатор для транзакций. Он должен быть уникальным для каждого продюсера в вашем приложении.

In [36]:
from confluent_kafka import Producer

# Функция для обработки успешной доставки сообщений
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

# Конфигурация продюсера
conf = {
    'bootstrap.servers': 'localhost:9092',  # Адрес Kafka брокера
    'client.id': 'my-producer',              # Идентификатор клиента
    'enable.idempotence': True,               # Включение идемпотентного режима
    'acks': 'all',                            # Подтверждение от всех реплик
    'retries': 5,                             # Количество попыток повторной отправки
    'transactional.id': 'my-transactional-id'  # Идентификатор транзакции
}

# Создание экземпляра продюсера
producer = Producer(conf)

# Инициализация транзакции
producer.init_transactions()

try:
    # Начало транзакции
    producer.begin_transaction()

    # Отправка сообщений
    for i in range(4):
        key = f'key-{i}'                # Ключ сообщения (опционально)
        value = f'Hello Kafka {i}'      # Значение сообщения
        producer.produce('my-topic', key=key, value=value, callback=delivery_report)

        # Обработка асинхронных операций
        producer.poll(0)  # Проверка на наличие ошибок и вызов обратных функций

    # Завершение транзакции
    producer.commit_transaction()

except Exception as e:
    print(f'Error while producing: {e}')
    # Отмена транзакции в случае ошибки
    producer.abort_transaction()

finally:
    # Ожидание отправки всех сообщений
    producer.flush()


%4|1729763253.160|GETPID|my-producer#producer-16| [thrd:main]: Failed to acquire transactional PID from broker TxnCoordinator/1: Broker: Not coordinator: retrying


Message delivered to my-topic [0] at offset 13
Message delivered to my-topic [0] at offset 14
Message delivered to my-topic [0] at offset 15
Message delivered to my-topic [0] at offset 16


AVRO

Avro-файл — это бинарный файл, который используется для хранения данных, сериализованных в формате Avro. Он состоит из двух основных частей:

Заголовок (Header): содержит метаданные и схему данных, которая описывает формат записей в файле.
Тело (Data Blocks): содержит сами данные (записи), закодированные в соответствии с указанной схемой.

In [28]:
#Структура Avro-файла
# 1. Заголовок (Header)
# Каждый Avro-файл начинается с заголовка, который включает в себя:

# Магическое число (magic): первые 4 байта файла — это последовательность байт
#  0x4F 0x62 0x6A 0x01 (или строка "Obj\x01"). Это используется для идентификации файла 
# как Avro-файл.
#Метаданные (Metadata): хранит JSON-объект с информацией о схеме (например, "avro.schema") 
# и другую служебную информацию.
#Синхронизационный маркер (Sync Marker): случайная последовательность из 16 байт, которая используется для синхронизации при чтении файла блоками.

In [62]:
## работа с avro 
import fastavro

# Определение схемы
schema = {
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"}
    ]
}

# Данные, которые будут сериализованы
records = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 30}
]

# Запись данных в Avro-файл
with open('test.avro', 'wb') as out:
    fastavro.writer(out, schema, records)


In [63]:
import fastavro

# Чтение Avro-файла
with open('test.avro', 'rb') as f:
    reader = fastavro.reader(f)
    for record in reader:
        print(record)


{'name': 'Alice', 'age': 25}
{'name': 'Bob', 'age': 30}


In [36]:
from uuid import uuid4
str(uuid4())

'11d10a2c-1188-48bf-9540-0a866c711adb'